# A place for me to learn and fiddle and try stuff

## First looking to incorporate height and lat long data.
Want to eyeball the data first to understand waypoint naming - does notebook agree with gpx files.
If it does load gpx files in a similar way to observations and then zip them together using the waypoint number

In [70]:
import pandas as pd

observations_df = pd.read_csv("../../data/processed/observations.csv")
observations_df.head()


,sample_point_id,observation_date,garmin_waypoint_id,garmin_grid_ref,photo_up_id,photo_down_id,wetness_estimate,canopy_cover_estimate,species_identified
0,9,2019-02-01,1,TL6787633142,P2010002,P2010003,1.0,95,"['Mercurialis perennis', 'Urtica dioica', 'Sil..."
1,59,2019-02-01,2,TL6787433118,P2010004,P2010005,1.0,65,"['Silene dioica', 'Galium aparine', 'Glechoma ..."
2,46,2019-02-01,3,TL6787933089,P2010006,P2010007,1.0,45,"['Glechoma hederacea', 'Galium aparine', 'Sile..."
3,30,2019-02-01,4,TL6787633061,P2010008,P2010009,1.0,0,"['Urtica dioica', 'Rubus fruticosus agg.', 'Gl..."
4,60,2019-02-01,5,TL6790833003,P2010010,P2010011,1.0,0,['Triticum aestivum']


Looks like there's a cross reference to waypoint id

In [71]:
observations_df.iloc[0:1]

,sample_point_id,observation_date,garmin_waypoint_id,garmin_grid_ref,photo_up_id,photo_down_id,wetness_estimate,canopy_cover_estimate,species_identified
0,9,2019-02-01,1,TL6787633142,P2010002,P2010003,1.0,95,"['Mercurialis perennis', 'Urtica dioica', 'Sil..."


Eyeball Waypoint 1
```xml
<wpt lat="51.971350" lon="0.442538">
    <ele>75.789368</ele>
    <time>2019-02-01T09:02:24Z</time>
    <name>001</name>
    <sym>Flag, Blue</sym>
</wpt>
```

https://gridreferencefinder.com/os.php#gr=TL6787633142|TL6787633142|1


![location summary](wp1_location.png)

10 figure grid reference, TL6787633142, 1m square
Lat long,5 decimal place, 1 m accuracy

How were positions derived. Garmin records lat long, OS grid reference read off garmin and recorded but not kept in GPX file.





In [72]:
observations_df['observation_date'] = pd.to_datetime(observations_df['observation_date'])
observations_df[observations_df['garmin_waypoint_id'] == 1].sort_values('observation_date')


,sample_point_id,observation_date,garmin_waypoint_id,garmin_grid_ref,photo_up_id,photo_down_id,wetness_estimate,canopy_cover_estimate,species_identified
0,9,2019-02-01,1,TL6787633142,P2010002,P2010003,1.0,95,"['Mercurialis perennis', 'Urtica dioica', 'Sil..."
32,3,2019-02-02,1,TL6778733146,P2020001,P2020002,1.0,5,"['Urtica dioica', 'Rubus fruticosus agg.', 'Ga..."
45,63,2019-06-20,1,TL6769933268,P6200001,P6200002,2.0,2,"['Filipendula ulmaria', 'Galium aparine', 'Arr..."
93,49,2020-06-19,1,TL6796933091,P6190001,P6190002,2.0,95,"['Urtica dioica', 'Galium aparine', 'Adoxa mos..."
102,42,2020-06-20,1,TL0793733099,P6200002,P6200003,5.0,65,['']
136,51,2020-06-21,1,TL6773233235,P6210001,P6210002,2.0,25,"['Mentha aquatica', 'Holcus lanatus', 'Glechom..."
141,5,2021-06-20,1,TL6809333054,P6200001,P6200002,2.0,45,"['Galium aparine', 'Rubus fruticosus agg.', 'G..."
169,45,2021-06-21,1,TL6770133235,P6210057,P6210058,3.0,55,"['Urtica dioica', 'Mercurialis perennis', 'Gal..."
189,49,2022-06-24,1,TL6796833089,P6240001,P6240002,3.0,80,"['Veronica beccabunga', 'Glechoma hederacea', ..."
237,49,2023-06-22,1,TL6796933086,P6220001,P6220002,4.0,60,"['Veronica beccabunga', 'Glechoma hederacea', ..."


Looks like waypoint ids vary, I guess sample point must be the index

In [73]:
observations_df['observation_date'] = pd.to_datetime(observations_df['observation_date'])
sample_point_id = 1
filtered_dates = observations_df[observations_df['sample_point_id'] == sample_point_id]['observation_date']

if not filtered_dates.empty:
    print(f"Observation dates for sample_point_id {sample_point_id}:")
    print(filtered_dates)
else:
    print(f"No observations found for sample_point_id {sample_point_id}")

Observation dates for sample_point_id 1:
43    2019-02-02
46    2019-06-20
116   2020-06-20
164   2021-06-20
224   2022-06-26
265   2023-06-24
330   2024-06-22
Name: observation_date, dtype: datetime64[ns]


Consistent. Waypoint id changed each time. Would be useful to pick out grid ref into separate easting and northing. Then can see how much it's varying. It would become plottable.

Looks like to get elevation would need to link observation date, and waypoint id file, to pick off the right id and get the elevation.

Next question - do the data wrangling in dataframe or initial conversion to csv. Instinct it to try latter.

So first job is to get waypoint to csv (or read xml into data frame... but then have to deal with dates...)

Quadrants remain constant, waypoints vary each time, but it looks like they don't repeat within years. Can zip waypoint information into observations. In observations need to consider waypoint number and year and pick out the relevant row. Presume with dataframes we could then split that all out into records, or categorical data to records could get elevation as well. But working in pandas rather than more native approach.

## Distance and Precision

Distance and Precision

Grid Reference Precision:<br>
A 6-figure grid reference (TL678331) gives you a location within a 1km square.<br>
A 10-figure grid reference (TL6787633142) gives you a location within a 1-meter square.<br>
Latitude/Longitude Precision:<br>
The number of decimal places in latitude and longitude determines the precision.<br>
In the example, 6 decimal places are used.<br>
Each decimal place roughly equates to:<br>
1 degree ≈ 111 km<br>
0.1 degree ≈ 11.1 km<br>
0.01 degree ≈ 1.11 km<br>
0.00001 degree ≈ 1.1 m.<br>
0.000001 degree ≈ 11 cm.<br>
Therefore, 6 decimal places provides accuracy to approximately 11 centimeters, which is very precise.<br>


Sorted out import of waypoint data into "../../data/processed/gpx_observations.csv"

In [74]:
gpx_df = pd.read_csv("../../data/processed/gpx_observations.csv")
gpx_df.head()

,lat,lon,ele,time,name,sym,cmt,date
0,51.971350,0.442538,75.789368,2019-02-01T09:02:24Z,1,"Flag, Blue",NaN,2019-02-01
1,51.971138,0.442500,81.909317,2019-02-01T09:14:19Z,2,"Flag, Blue",NaN,2019-02-01
2,51.970874,0.442566,74.995232,2019-02-01T09:26:47Z,3,"Flag, Blue",NaN,2019-02-01
3,51.970619,0.442503,73.338715,2019-02-01T09:39:07Z,4,"Flag, Blue",NaN,2019-02-01
4,51.970119,0.442947,73.245712,2019-02-01T10:09:01Z,5,"Flag, Blue",NaN,2019-02-01


Create a waypoint_year index column in gpx and observations then we can combine

In [75]:
gpx_df["waypoint_date"] = gpx_df["name"].astype(str) + "_" + gpx_df["date"].astype(str)
gpx_df.head()


,lat,lon,ele,time,name,sym,cmt,date,waypoint_date
0,51.971350,0.442538,75.789368,2019-02-01T09:02:24Z,1,"Flag, Blue",NaN,2019-02-01,1_2019-02-01
1,51.971138,0.442500,81.909317,2019-02-01T09:14:19Z,2,"Flag, Blue",NaN,2019-02-01,2_2019-02-01
2,51.970874,0.442566,74.995232,2019-02-01T09:26:47Z,3,"Flag, Blue",NaN,2019-02-01,3_2019-02-01
3,51.970619,0.442503,73.338715,2019-02-01T09:39:07Z,4,"Flag, Blue",NaN,2019-02-01,4_2019-02-01
4,51.970119,0.442947,73.245712,2019-02-01T10:09:01Z,5,"Flag, Blue",NaN,2019-02-01,5_2019-02-01


And in observations

In [76]:
# 2. Create Combined Key in Observations DataFrame
observations_df["waypoint_date"] = observations_df["garmin_waypoint_id"].astype(str) + "_" + observations_df["observation_date"].astype(str)

In [77]:
observations_df.head()


,sample_point_id,observation_date,garmin_waypoint_id,garmin_grid_ref,photo_up_id,photo_down_id,wetness_estimate,canopy_cover_estimate,species_identified,waypoint_date
0,9,2019-02-01,1,TL6787633142,P2010002,P2010003,1.0,95,"['Mercurialis perennis', 'Urtica dioica', 'Sil...",1_2019-02-01
1,59,2019-02-01,2,TL6787433118,P2010004,P2010005,1.0,65,"['Silene dioica', 'Galium aparine', 'Glechoma ...",2_2019-02-01
2,46,2019-02-01,3,TL6787933089,P2010006,P2010007,1.0,45,"['Glechoma hederacea', 'Galium aparine', 'Sile...",3_2019-02-01
3,30,2019-02-01,4,TL6787633061,P2010008,P2010009,1.0,0,"['Urtica dioica', 'Rubus fruticosus agg.', 'Gl...",4_2019-02-01
4,60,2019-02-01,5,TL6790833003,P2010010,P2010011,1.0,0,['Triticum aestivum'],5_2019-02-01


In [78]:
# 3. Merge DataFrames
merged_df = pd.merge(observations_df, gpx_df[["lat", "lon", "ele", "cmt", "date", "waypoint_date"]], on="waypoint_date", how="left")


In [79]:
merged_df


,sample_point_id,observation_date,garmin_waypoint_id,garmin_grid_ref,photo_up_id,photo_down_id,wetness_estimate,canopy_cover_estimate,species_identified,waypoint_date,lat,lon,ele,cmt,date
0,9,2019-02-01,1,TL6787633142,P2010002,P2010003,1.0,95,"['Mercurialis perennis', 'Urtica dioica', 'Sil...",1_2019-02-01,51.971350,0.442538,75.789368,NaN,2019-02-01
1,59,2019-02-01,2,TL6787433118,P2010004,P2010005,1.0,65,"['Silene dioica', 'Galium aparine', 'Glechoma ...",2_2019-02-01,51.971138,0.442500,81.909317,NaN,2019-02-01
2,46,2019-02-01,3,TL6787933089,P2010006,P2010007,1.0,45,"['Glechoma hederacea', 'Galium aparine', 'Sile...",3_2019-02-01,51.970874,0.442566,74.995232,NaN,2019-02-01
3,30,2019-02-01,4,TL6787633061,P2010008,P2010009,1.0,0,"['Urtica dioica', 'Rubus fruticosus agg.', 'Gl...",4_2019-02-01,51.970619,0.442503,73.338715,NaN,2019-02-01
4,60,2019-02-01,5,TL6790833003,P2010010,P2010011,1.0,0,['Triticum aestivum'],5_2019-02-01,51.970119,0.442947,73.245712,NaN,2019-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,63,2024-06-22,53,TL6766933264,P6220039,P6220040,2.0,45,"['Holcus lanatus', 'Poa trivialis', 'Mentha aq...",53_2024-06-22,51.972498,0.440033,78.009674,NaN,2024-06-22
329,16,2024-06-22,54,TL6769833294,P6220041,P6220042,2.0,60,"['Carduus crispus', 'Pulicaria dysenterica', '...",54_2024-06-22,51.972769,0.440033,76.182129,NaN,2024-06-22
330,1,2024-06-22,55,TL6773033265,P6220043,P6220044,2.0,65,"['Arrhenatherum elatius', 'Silene dioica', 'Me...",55_2024-06-22,51.972499,0.440486,76.922234,NaN,2024-06-22
331,51,2024-06-22,56,TL6772933234,P6220045,P6220046,3.5,5,"['Urtica dioica', 'Rubus fruticosus agg.', 'An...",56_2024-06-22,51.972222,0.440456,82.262337,NaN,2024-06-22


## Different tyes of analysis - first thoughts

### Data check
Variation in location of quadrants each year. Systematic shift?\
Agreement between OS grid and lat long\
Consistency of change in elevation

### Understanding the data
Getting a feel for changes over space and time of:\
vegetation\
canopy\
elevation

### Modelling / learning from the data
Traditional stats (approach tbd):
Look into modelling:\
wetness (dependent variable)\
canopy (dependent variable)?
most important predictor plants

Computer vision model:\
Look at canopy photos vs canopy estimate\
Recognising plants in ground photos\

### Enhancing the data
What about finding meteorological data to consider amount of rain before the survey period each year
